<a href="https://colab.research.google.com/github/newton143/Econ8310/blob/master/NeuralNetwork_Lesson7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For reading data
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# For visualizing
import plotly.express as px

# For model building
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class CustomMNIST(Dataset):
    def __init__(self, url):
        # read in our raw data from the hosting URL
        self.raw_data = pd.read_csv(url)

    # return the length of the complete data set
    #   to be used in internal calculations for pytorch
    def __len__(self):
        return self.raw_data.shape[0]

    # retrieve a single record based on index position `idx`
    def __getitem__(self, idx):
        # extract the image separate from the label
        image = self.raw_data.iloc[idx, 1:].values.reshape(1, 28, 28)
        # Specify dtype to align with default dtype used by weight matrices
        image = torch.tensor(image, dtype=torch.float32)
        # extract the label
        label = self.raw_data.iloc[idx, 0]

        # return the image and its corresponding label
        return image, label

In [3]:
# Load our data into memory
train_data = CustomMNIST("https://github.com/dustywhite7/Econ8310/raw/master/DataSets/mnistTrain.csv")
test_data = CustomMNIST("https://github.com/dustywhite7/Econ8310/raw/master/DataSets/mnistTest.csv")

# Create data feed pipelines for modeling
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
# Check that our data look right when we sample

idx=1
print(f"This image is labeled a {train_data.__getitem__(idx)[1]}")
px.imshow(train_data.__getitem__(idx)[0].reshape(28, 28))

This image is labeled a 9


In [7]:
class FirstNet(nn.Module):
    def __init__(self):
      # We define the components of our model here
      super(FirstNet, self).__init__()
      # Function to flatten our image
      self.flatten = nn.Flatten()
      # Create the sequence of our network
      self.linear_relu_model = nn.Sequential(
            # Add a linear output layer w/ 10 perceptrons
            nn.LazyLinear(10),
        )

    def forward(self, x):
      # We construct the sequencing of our model here
      x = self.flatten(x)
      # Pass flattened images through our sequence
      output = self.linear_relu_model(x)

      # Return the evaluations of our ten
      #   classes as a 10-dimensional vector
      return output


In [14]:
# Create an instance of our model
model = FirstNet()

In [13]:
# Define some training parameters
learning_rate = 1e-2
batch_size = 64
epochs = 20

# Define our loss function
#   This one works for multiclass problems
loss_fn = nn.CrossEntropyLoss()

In [10]:
# Define some training parameters
learning_rate = 1e-2
batch_size = 64
epochs = 20

# Define our loss function
#   This one works for multiclass problems
loss_fn = nn.CrossEntropyLoss()

In [15]:
# Build our optimizer with the parameters from
#   the model we defined, and the learning rate
#   that we picked
optimizer = torch.optim.SGD(model.parameters(),
     lr=learning_rate)

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode
    # important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    # Loop over batches via the dataloader
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation and looking for improved gradients
        loss.backward()
        optimizer.step()
        # Zeroing out the gradient (otherwise they are summed)
        #   in preparation for next round
        optimizer.zero_grad()

        # Print progress update every few loops
        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode
    # important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures
    # that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations
    # and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Printing some output after a testing round
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
# Need to repeat the training process for each epoch.
#   In each epoch, the model will eventually see EVERY
#   observations in the data
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.311963  [   64/ 5000]
loss: 2.226088  [  704/ 5000]
loss: 2.166538  [ 1344/ 5000]
loss: 2.043823  [ 1984/ 5000]
loss: 1.919156  [ 2624/ 5000]
loss: 1.897462  [ 3264/ 5000]
loss: 1.775369  [ 3904/ 5000]
loss: 1.721654  [ 4544/ 5000]
Test Error: 
 Accuracy: 75.4%, Avg loss: 1.677561 

Epoch 2
-------------------------------
loss: 1.667881  [   64/ 5000]
loss: 1.612344  [  704/ 5000]
loss: 1.650731  [ 1344/ 5000]
loss: 1.566091  [ 1984/ 5000]
loss: 1.425802  [ 2624/ 5000]
loss: 1.436833  [ 3264/ 5000]
loss: 1.346278  [ 3904/ 5000]
loss: 1.321608  [ 4544/ 5000]
Test Error: 
 Accuracy: 79.9%, Avg loss: 1.312569 

Epoch 3
-------------------------------
loss: 1.296955  [   64/ 5000]
loss: 1.271047  [  704/ 5000]
loss: 1.364068  [ 1344/ 5000]
loss: 1.301675  [ 1984/ 5000]
loss: 1.145915  [ 2624/ 5000]
loss: 1.171075  [ 3264/ 5000]
loss: 1.102338  [ 3904/ 5000]
loss: 1.100435  [ 4544/ 5000]
Test Error: 
 Accuracy: 81.5%, Avg loss: 1.099814 

Epoc

In [20]:
# Decide if we are loading for predictions or more training
model.eval()
# - or -
# model.train()

# Make predictions
pred = model(test_data.__getitem__(1)[0]).argmax()
truth = test_data.__getitem__(1)[1]
print(f"This image is predicted to be a {pred}, and is labeled as {truth}")

This image is predicted to be a 6, and is labeled as 6
